In [1]:
import os
import pandas as pd
import re
import urllib.request
from urllib.error import HTTPError

# Setup

target_website = 'google.com' # the website to look for when filtering down hrefs

cwd = os.getcwd()
#m_path = cwd if 'manuscript-object' not in cwd else f'{cwd}/../'
m_path = cwd if 'manuscript-object' in cwd else f'{cwd}/manuscript-object/'
fieldnotes_path = f'{m_path}/fieldnotes/FA18+other-fieldnotes-list+links.csv'
out_path = f'{m_path}/fieldnotes/fieldnotes_hrefs.csv'

url_column = 'full-html'
hrefs_column = 'references'

re_href = re.compile(r'<a href="(https?://.*?)".*?>') # regex to find http hrefs in a block of html
#re_href = re.compile(r'<a href="(.*?)".*?>') # regex to find hrefs in a block of html (not necessarily web links)

In [2]:
# Script:
def find_hrefs_from_url(url:str, regex, target='') -> str:
    try:
        page = urllib.request.urlopen(url) # open url
    except HTTPError:
        return 'url not found'
    except ValueError:
        return 'not a valid web address'
    
    page_text = page.read().decode('utf-8') # decode to text
    
    matches = [href for href in regex.findall(page_text) if target in href] # get hrefs linking to target
    
    if not matches:
        return 'none'     # if target nowhere in that url, return 'none'
    else:
        return ','.join(matches)  # otherwise return a string containing each href separated by commas

In [3]:
fieldnotes = pd.read_csv(fieldnotes_path)

In [7]:
fieldnotes[hrefs_column] = fieldnotes[url_column].apply(lambda url: find_hrefs_from_url(url, re_href, target=target_website))

In [6]:
#temp = [len(url) for url in fieldnotes[hrefs_column]]
#fieldnotes[url_column][temp.index(max(temp))]


['url not found',
 'url not found',
 'url not found',
 'url not found',
 'url not found']

In [99]:
fieldnotes.to_csv(out_path, index=False)